# **Feature Engineering - previous_application**

### **Importar Bibliotecas**

In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1702430560900_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg
from pyspark.sql.functions import col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Importar tabela 'previous_application'.**

In [3]:
spark = SparkSession.builder.appName('Feature Engineering').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1702430560900_0003/container_1702430560900_0003_01_000001/pyspark.zip/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

In [4]:
file='previous_application.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
S3_DATA_SOURCE_PATH = 'pod-academy-analise-de-credito-para-fintech/dados'
S3_DATA_OUTPUT_PATH = 'pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
output = 'abt_previous_application'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
full_path_source = 's3://{bkt}/{file}'.format(bkt=S3_DATA_SOURCE_PATH,file=file)
print(full_path_source)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/dados/previous_application.csv

In [8]:
previous_application_00 = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_source)
previous_application_00.registerTempTable("previous_application_00")
previous_application_00.cache()
qtd=previous_application_00.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 1670214
/mnt/yarn/usercache/livy/appcache/application_1702430560900_0003/container_1702430560900_0003_01_000001/pyspark.zip/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.

In [9]:
previous_application_00.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURP

In [10]:
# +----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
# |SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURPOSE|NAME_CONTRACT_STATUS|DAYS_DECISION|NAME_PAYMENT_TYPE    |CODE_REJECT_REASON|NAME_TYPE_SUITE|NAME_CLIENT_TYPE|NAME_GOODS_CATEGORY|NAME_PORTFOLIO|NAME_PRODUCT_TYPE|CHANNEL_TYPE           |SELLERPLACE_AREA|NAME_SELLER_INDUSTRY|CNT_PAYMENT|NAME_YIELD_GROUP|PRODUCT_COMBINATION     |DAYS_FIRST_DRAWING|DAYS_FIRST_DUE|DAYS_LAST_DUE_1ST_VERSION|DAYS_LAST_DUE|DAYS_TERMINATION|NFLAG_INSURED_ON_APPROVAL|
# +----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
# |2030495   |271877    |Consumer loans    |1730.43    |17145.0        |17145.0   |0.0             |17145.0        |SATURDAY                  |15                     |Y                          |1                     |0.0              |0.18283180324152784  |0.8673361522198731      |XAP                   |Approved            |-73          |Cash through the bank|XAP               |null           |Repeater        |Mobile             |POS           |XNA              |Country-wide           |35              |Connectivity        |12.0       |middle          |POS mobile with interest|365243.0          |-42.0         |300.0                    |-42.0        |-37.0           |0.0                      |
# |2802425   |108129    |Cash loans        |25188.615  |607500.0       |679671.0  |null            |607500.0       |THURSDAY                  |11                     |Y                          |1                     |null             |null                 |null                    |XNA                   |Approved            |-164         |XNA                  |XAP               |Unaccompanied  |Repeater        |XNA                |Cash          |x-sell           |Contact center         |-1              |XNA                 |36.0       |low_action      |Cash X-Sell: low        |365243.0          |-134.0        |916.0                    |365243.0     |365243.0        |1.0                      |
# |2523466   |122040    |Cash loans        |15060.735  |112500.0       |136444.5  |null            |112500.0       |TUESDAY                   |11                     |Y                          |1                     |null             |null                 |null                    |XNA                   |Approved            |-301         |Cash through the bank|XAP               |Spouse, partner|Repeater        |XNA                |Cash          |x-sell           |Credit and cash offices|-1              |XNA                 |12.0       |high            |Cash X-Sell: high       |365243.0          |-271.0        |59.0                     |365243.0     |365243.0        |1.0                      |
# |2819243   |176158    |Cash loans        |47041.335  |450000.0       |470790.0  |null            |450000.0       |MONDAY                    |7                      |Y                          |1                     |null             |null                 |null                    |XNA                   |Approved            |-512         |Cash through the bank|XAP               |null           |Repeater        |XNA                |Cash          |x-sell           |Credit and cash offices|-1              |XNA                 |12.0       |middle          |Cash X-Sell: middle     |365243.0          |-482.0        |-152.0                   |-182.0       |-177.0          |1.0                      |
# |1784265   |202054    |Cash loans        |31924.395  |337500.0       |404055.0  |null            |337500.0       |THURSDAY                  |9                      |Y                          |1                     |null             |null                 |null                    |Repairs               |Refused             |-781         |Cash through the bank|HC                |null           |Repeater        |XNA                |Cash          |walk-in          |Credit and cash offices|-1              |XNA                 |24.0       |high            |Cash Street: high       |null              |null          |null                     |null         |null            |null                     |
# +----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
previous_application_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- SK_ID_PREV: string (nullable = true)
 |-- SK_ID_CURR: string (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- AMT_ANNUITY: string (nullable = true)
 |-- AMT_APPLICATION: string (nullable = true)
 |-- AMT_CREDIT: string (nullable = true)
 |-- AMT_DOWN_PAYMENT: string (nullable = true)
 |-- AMT_GOODS_PRICE: string (nullable = true)
 |-- WEEKDAY_APPR_PROCESS_START: string (nullable = true)
 |-- HOUR_APPR_PROCESS_START: string (nullable = true)
 |-- FLAG_LAST_APPL_PER_CONTRACT: string (nullable = true)
 |-- NFLAG_LAST_APPL_IN_DAY: string (nullable = true)
 |-- RATE_DOWN_PAYMENT: string (nullable = true)
 |-- RATE_INTEREST_PRIMARY: string (nullable = true)
 |-- RATE_INTEREST_PRIVILEGED: string (nullable = true)
 |-- NAME_CASH_LOAN_PURPOSE: string (nullable = true)
 |-- NAME_CONTRACT_STATUS: string (nullable = true)
 |-- DAYS_DECISION: string (nullable = true)
 |-- NAME_PAYMENT_TYPE: string (nullable = true)
 |-- CODE_REJECT_REASON: string (nullable = true)


### **Comentário**

- Cada "SK_ID_PREV" se refere a um único "SK_ID_CURR".

- Um "SK_ID_CURR" pode corresponder a diferentes "SK_ID_PREV".

- Ou seja, podemos referenciar as variáveis criadas com a tabela "application", assim como, a tabela "previous_application".

In [12]:
# previous_application_00.createOrReplaceTempView("previous_application_00")

# previous_application_00 = spark.sql('''
# SELECT
#   *
# FROM
#   previous_application_00
# WHERE
#   SK_ID_CURR = '405549'
# ''')

# previous_application_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# # Contar valores distintos da variável SK_ID_CURR
# qtde_distinct_sk_id_curr = spark.sql('''
# SELECT
#   COUNT(DISTINCT SK_ID_CURR) AS qtde_distinct_sk_id_curr
# FROM
#   previous_application_00
# ''')

# # Exibir a quantidade de valores distintos
# qtde_distinct_sk_id_curr.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# **Criação de variáveis preditivas**

## **previous_application**

In [14]:
# Definir funções de agregações desejadas
agg = ['SUM', 'MAX', 'MIN', 'AVG']

var_categoricas = ['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',\
                   'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',\
                   'NAME_CLIENT_TYPE', 'NAME_PRODUCT_TYPE']

var_numericas = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', \
                 'HOUR_APPR_PROCESS_START',\
                 'DAYS_DECISION','SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 
                 'DAYS_FIRST_DUE','DAYS_LAST_DUE', 'DAYS_TERMINATION']

# Definir flags de meses
# meses_flag = ['ultimos_3_meses_flag', 'ultimos_6_meses_flag', 'ultimos_9_meses_flag', 'ultimos_12_meses_flag']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando variáveis explicativas de primeira camada**

- usar média, mínimo, soma, máximo
- considerar variáveis categóricas e variáveis numéricas

### **Gerar variáveis.**

In [15]:
# Importar SparkSession
from pyspark.sql import SparkSession

# Criar uma sessão do Spark
spark = SparkSession.builder.appName("AdaptacaoCodigo").getOrCreate()

df_original_spark = previous_application_00

# Inicializar o contador
contador_resultados = 0

# Criar lista para armazenar os resultados
resultados = []

for ag in agg:
    for var_cat in var_categoricas:
        # Filtrar valores distintos não nulos
        distinct_values = df_original_spark.select(var_cat).distinct().na.drop(subset=[var_cat]).rdd.flatMap(lambda x: x).collect()

        for dom_cat_value in distinct_values:
            for var_numerica in var_numericas:
                resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_{var_numerica.lower().replace(' ', '_').replace('-', '_')}_{var_numerica.lower().replace(' ', '_').replace('-', '_')}_{var_cat.lower()}_{dom_cat_value.lower().replace(' ', '_').replace('-', '')},"

                # Adicionar o resultado à lista
                resultados.append(resultado)

                # for mes_flag in meses_flag:
                #     resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' AND {mes_flag} = 1 THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_U{mes_flag.split('_')[1]}M_{var_numerica.lower()[:5]}_{var_cat.lower()[:4]}_{dom_cat_value.lower()[:3].replace(' ', '_')},"

                #     # Adicionar o resultado à lista
                #     resultados.append(resultado)

                # Incrementar o contador
                contador_resultados += 1

# Printar o total de combinações
print(f"Total de combinações: {contador_resultados}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total de combina??es: 1820

In [16]:
resultados[396]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"ROUND(SUM(CASE WHEN NAME_CLIENT_TYPE = 'Repeater' THEN DAYS_DECISION ELSE NULL END), 2) as sum_days_decision_days_decision_name_client_type_repeater,"

In [17]:
# Inicializa a lista para armazenar os resultados
resultados_formatados = []

# Se há pelo menos um resultado na lista
if resultados:
    # Remove a vírgula do último item
    resultados[-1] = resultados[-1].rstrip(',')

    # Adiciona os resultados formatados à lista
    resultados_formatados.extend(resultados)

    # Une a lista em uma string usando uma quebra de linha como separador
    resultados_str = '\n'.join(resultados_formatados)
    # print(resultados_str)
else:
    resultados_str = "A lista de resultados está vazia."

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Colar variáveis criadas no código PySpark-SQL.**

In [34]:
previous_application_02 = spark.sql(f'''
SELECT
  SK_ID_CURR as SK_ID_CURR_previous_application,
  {resultados_str}
FROM
  previous_application_00
GROUP BY
  SK_ID_CURR
''')

previous_application_02.createOrReplaceTempView("previous_application_02")
previous_application_02.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

338857

In [35]:
print(len(previous_application_02.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1821

In [36]:
# previous_application_02.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Salvando tabela no diretório do S3.**

In [37]:
# Número desejado de partições
num_partitions = 5

# Garanta que a coluna "id" seja numérica para garantir uma distribuição uniforme
previous_application_02 = previous_application_02.withColumn("SK_ID_CURR_previous_application",\
                                                             col("SK_ID_CURR_previous_application")\
                                                             .cast("int"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# Save to parquet
path_output = os.path.join('s3://',S3_DATA_OUTPUT_PATH,output)
print(path_output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_previous_application

In [39]:
# Salve cada partição em um arquivo Parquet separado
for i in range(num_partitions):
    num_cols_i = int(((len(previous_application_02.columns))/num_partitions*(i-1))+1)
    num_cols_f = int((len(previous_application_02.columns))/num_partitions*i)
    
    previous_application_02_part = previous_application_02.select(*["SK_ID_CURR_previous_application"]+[previous_application_02.columns[i] for i in range(num_cols_i,num_cols_f)])
    previous_application_02_part.coalesce(1).write.parquet(f'{path_output}_0{i}',mode='overwrite')
    
    # Obtenha as dimensões do DataFrame particionado
    num_rows = previous_application_02_part.count()
    num_cols = len(previous_application_02_part.columns)
    
    # Imprima informações sobre o arquivo criado
    print(f"Arquivo {i} - Caminho: {path_output}_0{i}, Linhas: {num_rows}, Colunas: {num_cols}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Arquivo 0 - Caminho: s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_previous_application_00, Linhas: 338857, Colunas: 364
Arquivo 1 - Caminho: s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_previous_application_01, Linhas: 338857, Colunas: 364
Arquivo 2 - Caminho: s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_previous_application_02, Linhas: 338857, Colunas: 364
Arquivo 3 - Caminho: s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_previous_application_03, Linhas: 338857, Colunas: 364
Arquivo 4 - Caminho: s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/02-Previous-Application/abt_previous_application_04, Linhas: 338857, Colunas: 364

In [24]:
# # Save to parquet
# path_output = os.path.join('s3://',S3_DATA_OUTPUT_PATH,output)
# print(path_output)
# previous_application_02.coalesce(5).write.\
# parquet(path_output,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# abt_previous_application=sqlContext.read.format("parquet").load('s3://',S3_DATA_OUTPUT_PATH,output)
# abt_previous_application.registerTempTable("abt_previous_application")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# abt_previous_application.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…